In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import os
import pickle




In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [3]:
if torch.cuda.is_available():
    device = "cuda:0"

In [4]:
#model = GPT2LMHeadModel.from_pretrained('gpt2-xl', cache_dir='/bigstor/zsarwar/models/cache/')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large', cache_dir='/bigstor/zsarwar/models/cache/')


In [10]:
len(tokenizer("Joe has a sister, played basketball and now lives in China and Jack lives in Santiago. The capital of Joe's country of residence is")['input_ids'])

27

In [6]:
model = model.to(device)

In [7]:
# Loading a pickle object
input_dataset = "/home/zsarwar/NLP/NLP_Repo/NLP_Project/Results/Incorrect/GPT2-XL/predictions_small_complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2XL.pickle"
with open(input_dataset, 'rb') as in_file:
    s_l = pickle.load(in_file)

curr_file = ""

In [8]:
#This is for if you are continuing predictions from the above dataset
#Don't run cell below if you run this
#Loading currently saved prediction 
#Current saved pickle file for all prompts already corrected
curr_file = "/home/zsarwar/NLP/NLP_Repo/NLP_Project/Results/Corrected/GPT2-XL/predictions_small_complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2XL.pickle"

with open(curr_file,'rb') as c_file:
    curr_ds = pickle.load(c_file)


# This is if you are starting work on this dataset

all_sents = curr_ds['all_sents']
all_preds = curr_ds['all_preds']
all_true_labels = curr_ds['all_true_labels']
all_pred_labels_correct = curr_ds['all_pred_labels_correct']

current_index = len(all_sents)

In [8]:
# This is if you are starting work on this dataset
#Don't run cell above if you run this
all_sents = []
all_preds = []
all_true_labels = []
all_pred_labels_correct = []
all_curr_pred_label_val = []
current_index = 0

In [8]:
# Start of a 'loop'

In [8]:
pred_correct = False

In [9]:
curr_sent = s_l['sentences'][current_index]

In [10]:
curr_sent

"Sebastian has a sister , played basketball and now lives in France and has travelled to Helsinki , Jakarta and Warsaw . The capital of Sebastian's country is"

In [11]:
curr_pred = s_l['predictions'][current_index]
curr_pred

' Warsaw, where he is a student at the University of'

In [12]:
curr_label = s_l['labels'][current_index]
all_true_labels.append(curr_label)
curr_label

'paris'

In [40]:
new_prompt = "Jack has a sister, played basketball and now lives in Indonesia and Jake lives in Paris. The capital of the country Jack lives in is"

encoded_prompt = tokenizer(new_prompt,add_special_tokens=False, return_attention_mask=False, return_tensors='pt').input_ids.to(device)
outputs = model.generate(encoded_prompt, do_sample=False, max_new_tokens=10, output_scores=True, return_dict_in_generate=True)
answer_start_idx = encoded_prompt.size()[1]
gen_answer = tokenizer.decode(outputs.sequences[0][answer_start_idx:], skip_special_tokens=True)
gen_answer

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Paris.\n\nThe show is set in Paris'

In [16]:
if(curr_label in gen_answer.lower()):
    print("Correct prediction")
    current_index+=1
    pred_correct = True
    
else:
    print("Incorrect prediction")
    current_index+=1
    pred_correct=False

if(pred_correct):
# Save all prompts that generate the correct answer in the format of the input pickle file
    all_pred_labels_correct.append("Correct")
else:
    all_pred_labels_correct.append("Incorrect")


all_sents.append(new_prompt)
all_preds.append(gen_answer)

Correct prediction


In [17]:
all_sents


['Rowan has a sister and now lives in France and has travelled to Santiago, Helsinki and Warsaw. The capital of the country in which Rowan lives is',
 'Rowan lives in France and has travelled to Beijing , Warsaw and Helsinki . The capital of the country Rowan lives in is',
 'Rowan lives in France and has travelled to Helsinki , Beijing and Warsaw . The capital of the country Rowan lives in is',
 "Rowan has a sister and now lives in France and has travelled to Helsinki, Beijing and Warsaw. The capital of Rowan's country of residence is the city of",
 "Sebastian has a sister and now lives in France and has travelled to Helsinki , Jakarta and Warsaw . The capital of the country Sebastian's lives in is the city of",
 "Sebastian has a sister, played basketball and now lives in France and has travelled to Helsinki, Jakarta and Warsaw . The capital of Sebastian's country is"]

In [18]:

all_preds 

[" called Paris.\n\nRowan's mother, who is in",
 ' Paris.\n\nShe has been a member of the Chinese Communist',
 ' Paris.\n\nThe first time I met Rowan was in',
 " Paris.\n\nRowan's father, who is a retired teacher, is a retired police officer",
 ' Warsaw.\n\nSebastian is a very good friend of the author and has been for',
 ' Paris.\n\nSebastian is a big fan of the band "The Killers"']

In [19]:
all_true_labels

['paris', 'paris', 'paris', 'paris', 'paris', 'paris', 'paris', 'paris']

In [20]:
all_sents

['Rowan has a sister and now lives in France and has travelled to Santiago, Helsinki and Warsaw. The capital of the country in which Rowan lives is',
 'Rowan lives in France and has travelled to Beijing , Warsaw and Helsinki . The capital of the country Rowan lives in is',
 'Rowan lives in France and has travelled to Helsinki , Beijing and Warsaw . The capital of the country Rowan lives in is',
 "Rowan has a sister and now lives in France and has travelled to Helsinki, Beijing and Warsaw. The capital of Rowan's country of residence is the city of",
 "Sebastian has a sister and now lives in France and has travelled to Helsinki , Jakarta and Warsaw . The capital of the country Sebastian's lives in is the city of",
 "Sebastian has a sister, played basketball and now lives in France and has travelled to Helsinki, Jakarta and Warsaw . The capital of Sebastian's country is"]

In [21]:
# Stop and save your current progress

In [22]:
new_prompts_preds = {}
new_prompts_preds['all_sents'] = all_sents
new_prompts_preds['all_preds'] = all_preds
new_prompts_preds['all_true_labels'] = all_true_labels
new_prompts_preds['all_pred_labels_correct'] = all_pred_labels_correct

In [23]:
if(curr_file != ""):
    out_file = curr_file
else:
    out_dir = '/'.join(input_dataset.split('/')[0:-3])
    out_file = out_dir + '/' + 'Corrected/GPT2-XL/' + input_dataset.split('/')[-1]


In [24]:
out_file

'/home/zsarwar/NLP/NLP_Repo/NLP_Project/Results/Corrected/GPT2-XL/predictions_small_complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2XL.pickle'

In [25]:
with open(out_file, 'wb') as o_file:
    pickle.dump(new_prompts_preds, o_file)